In [1]:
import onnx
import onnxruntime as ort

model_path = "hair_classifier_v1.onnx"

# Opción 1: con onnx
model = onnx.load(model_path)
graph = model.graph

print("Inputs:")
for i in graph.input:
    print(i.name, i.type)

print("\nOutputs:")
for o in graph.output:
    print(o.name, o.type)

# Opción 2: con onnxruntime
session = ort.InferenceSession(model_path, providers=["CPUExecutionProvider"])

print("\nONNXRuntime outputs:")
for o in session.get_outputs():
    print(o.name, o.shape, o.type)


Inputs:
input tensor_type {
  elem_type: 1
  shape {
    dim {
      dim_param: "s77"
    }
    dim {
      dim_value: 3
    }
    dim {
      dim_value: 200
    }
    dim {
      dim_value: 200
    }
  }
}


Outputs:
output tensor_type {
  elem_type: 1
  shape {
    dim {
      dim_param: "s77"
    }
    dim {
      dim_value: 1
    }
  }
}


ONNXRuntime outputs:
output ['s77', 1] tensor(float)


In [2]:
import numpy as np
from PIL import Image
from io import BytesIO
from urllib import request

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    return Image.open(BytesIO(buffer))

def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    return img.resize(target_size, Image.NEAREST)

url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"

img = download_image(url)
img = prepare_image(img, (200, 200))

x = np.array(img).astype("float32") / 255.0

x_nchw = np.transpose(x, (2, 0, 1))
x_nchw = np.expand_dims(x_nchw, 0)

first_pixel_R = x_nchw[0, 0, 0, 0]
first_pixel_R


np.float32(0.23921569)

In [3]:
import numpy as np
import onnxruntime as ort
from PIL import Image
from io import BytesIO
from urllib import request

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    return Image.open(BytesIO(buffer))

def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    return img.resize(target_size, Image.NEAREST)

url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"

# Preprocesar imagen
img = download_image(url)
img = prepare_image(img, (200, 200))

x = np.array(img).astype("float32") / 255.0
x = np.transpose(x, (2, 0, 1))
x = np.expand_dims(x, 0)

# Cargar modelo ONNX
session = ort.InferenceSession("hair_classifier_v1.onnx", providers=["CPUExecutionProvider"])

input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

pred = session.run([output_name], {input_name: x})[0]
score = float(pred.flatten()[0])
score


1.9217668771743774